In [1]:
from Capsule_Keras import *
from keras import utils
from keras.datasets import mnist
from keras.models import Model
from keras.layers import *
from keras import backend as K

/opt/algor/gongxf/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#准备训练数据
batch_size = 128
num_classes = 10
img_rows, img_cols = 28, 28


In [60]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data('./mnist/mnist.npz')
f = np.load('./mnist/mnist.npz')
x_train, y_train = f['x_train'], f['y_train']
x_test, y_test = f['x_test'], f['y_test']

In [61]:
x_train.shape

(60000, 28, 28)

In [71]:
y_train.min()

0

In [70]:
utils.to_categorical(y_train, 10).shape

(60000, 10)

In [17]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [21]:
y_train.shape

(60000, 10)

In [18]:
x_train.shape

(60000, 28, 28, 1)

In [22]:
y_test.shape

(10000, 10)

In [23]:
x_test.shape

(10000, 28, 28, 1)

In [24]:
idx = list(range(len(x_test)))
np.random.shuffle(idx)

In [27]:
X_test = np.concatenate([x_test, x_test[idx]], 1)

In [28]:
X_test.shape

(10000, 56, 28, 1)

In [29]:
Y_test = np.vstack([y_test.argmax(1), y_test[idx].argmax(1)]).T

In [30]:
Y_test.shape

(10000, 2)

In [31]:
Y_test[0]

array([7, 4])

In [33]:
X_test = X_test[Y_test[:,0] != Y_test[:,1]] #确保两个数字不一样

In [34]:
X_test.shape

(8990, 56, 28, 1)

In [35]:
Y_test = Y_test[Y_test[:,0] != Y_test[:,1]]
Y_test.sort(axis=1) #排一下序，因为只比较集合，不比较顺序

In [36]:
Y_test.shape

(8990, 2)

In [37]:
input_image = Input(shape=(None,None,1))

In [38]:
input_image.shape

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(1)])

In [39]:
cnn = Conv2D(64, (3, 3), activation='relu')(input_image)
cnn = Conv2D(64, (3, 3), activation='relu')(cnn)
cnn = AveragePooling2D((2,2))(cnn)
cnn = Conv2D(128, (3, 3), activation='relu')(cnn)
cnn = Conv2D(128, (3, 3), activation='relu')(cnn)
cnn = GlobalAveragePooling2D()(cnn)
dense = Dense(128, activation='relu')(cnn)
output = Dense(10, activation='sigmoid')(dense)

In [42]:
model = Model(inputs=input_image, outputs=output)

In [43]:
model.compile(loss=lambda y_true,y_pred: y_true*K.relu(0.9-y_pred)**2 + 0.25*(1-y_true)*K.relu(y_pred-0.1)**2,
              optimizer='adam',
              metrics=['accuracy'])

In [44]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
average_pooling2d_1 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 128)   73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 128)   147584    
_________________________________________________________________
global_average_pooling2d_1 ( (None, 128)               0         
__________

In [51]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 68s 1ms/step - loss: 0.0027 - acc: 0.9615 - val_loss: 0.0020 - val_acc: 0.9712
Epoch 2/10
60000/60000 [==============================] - 65s 1ms/step - loss: 0.0017 - acc: 0.9744 - val_loss: 0.0013 - val_acc: 0.9810
Epoch 3/10
60000/60000 [==============================] - 65s 1ms/step - loss: 0.0013 - acc: 0.9812 - val_loss: 0.0015 - val_acc: 0.9787
Epoch 4/10
60000/60000 [==============================] - 65s 1ms/step - loss: 0.0010 - acc: 0.9845 - val_loss: 9.0154e-04 - val_acc: 0.9869
Epoch 5/10
60000/60000 [==============================] - 65s 1ms/step - loss: 8.6182e-04 - acc: 0.9867 - val_loss: 8.4880e-04 - val_acc: 0.9871
Epoch 6/10
60000/60000 [==============================] - 65s 1ms/step - loss: 7.6062e-04 - acc: 0.9882 - val_loss: 6.5613e-04 - val_acc: 0.9897
Epoch 7/10
60000/60000 [==============================] - 65s 1ms/step - loss: 6.5621e-04 - acc: 0.9900 - va

In [52]:
Y_pred = model.predict(X_test)

In [53]:
greater = np.sort(Y_pred, axis=1)[:,-2] > 0.5

In [54]:
Y_pred = Y_pred.argsort()[:,-2:] #取最高分数的两个类别
Y_pred.sort(axis=1) #排序，因为只比较集合

In [55]:
acc = 1.*(np.prod(Y_pred == Y_test, axis=1)).sum()/len(X_test)
print('CNN+Pooling，不考虑置信度的准确率为：%s'%acc)
acc = 1.*(np.prod(Y_pred == Y_test, axis=1)*greater).sum()/len(X_test)
print('CNN+Pooling，考虑置信度的准确率为：%s'%acc)

CNN+Pooling，不考虑置信度的准确率为：0.30912124582869854
CNN+Pooling，考虑置信度的准确率为：0.0967741935483871


## 胶囊网络

In [56]:
#搭建CNN+Capsule分类模型
input_image = Input(shape=(None,None,1))
cnn = Conv2D(64, (3, 3), activation='relu')(input_image)
cnn = Conv2D(64, (3, 3), activation='relu')(cnn)
cnn = AveragePooling2D((2,2))(cnn)
cnn = Conv2D(128, (3, 3), activation='relu')(cnn)
cnn = Conv2D(128, (3, 3), activation='relu')(cnn)
cnn = Reshape((-1, 128))(cnn)
capsule = Capsule(10, 16, 3, True)(cnn)
output = Lambda(lambda x: K.sqrt(K.sum(K.square(x), 2)), output_shape=(10,))(capsule)

model = Model(inputs=input_image, outputs=output)
model.compile(loss=lambda y_true,y_pred: y_true*K.relu(0.9-y_pred)**2 + 0.25*(1-y_true)*K.relu(y_pred-0.1)**2,
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 64)    640       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
average_pooling2d_2 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, None, None, 128)   73856     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, None, None, 128)   147584    
___________________________________________________________

In [57]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))



Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 78s 1ms/step - loss: 0.0111 - acc: 0.8608 - val_loss: 0.0029 - val_acc: 0.9772
Epoch 2/5
60000/60000 [==============================] - 78s 1ms/step - loss: 0.0020 - acc: 0.9833 - val_loss: 0.0017 - val_acc: 0.9858
Epoch 3/5
60000/60000 [==============================] - 78s 1ms/step - loss: 0.0013 - acc: 0.9888 - val_loss: 0.0013 - val_acc: 0.9897
Epoch 4/5
60000/60000 [==============================] - 78s 1ms/step - loss: 0.0010 - acc: 0.9911 - val_loss: 0.0011 - val_acc: 0.9885
Epoch 5/5
60000/60000 [==============================] - 85s 1ms/step - loss: 8.5773e-04 - acc: 0.9929 - val_loss: 9.3911e-04 - val_acc: 0.9915


In [58]:
Y_pred = model.predict(X_test) #用模型进行预测
greater = np.sort(Y_pred, axis=1)[:,-2] > 0.5 #判断预测结果是否大于0.5
Y_pred = Y_pred.argsort()[:,-2:] #取最高分数的两个类别
Y_pred.sort(axis=1) #排序，因为只比较集合

acc = 1.*(np.prod(Y_pred == Y_test, axis=1)).sum()/len(X_test)
print('CNN+Capsule，不考虑置信度的准确率为：%s'%acc)
acc = 1.*(np.prod(Y_pred == Y_test, axis=1)*greater).sum()/len(X_test)
print('CNN+Capsule，考虑置信度的准确率为：%s'%acc)

CNN+Capsule，不考虑置信度的准确率为：0.9314794215795328
CNN+Capsule，考虑置信度的准确率为：0.9314794215795328
